# Prepare data

In [2]:
import os
from IPython.display import display, Audio

movies = {
    # 'foreigner': 'https://vz-5b088fab-d61.b-cdn.net/063b9069-85f0-4d30-9fc0-837cda23820c/playlist.m3u8',
    'captive': 'https://vz-5b088fab-d61.b-cdn.net/0adfd35e-15b8-4be5-8a00-a09456553df0/playlist.m3u8',
    'sun': 'https://vz-5b088fab-d61.b-cdn.net/9918149c-c555-448c-b0d9-4f4515233807/playlist.m3u8',
    'cabarete': 'https://vz-5b088fab-d61.b-cdn.net/3763464d-a485-44f8-9579-e981307ea79d/playlist.m3u8',
    'boost': 'https://vz-5b088fab-d61.b-cdn.net/07444a03-2c2b-41e9-8ef6-b2011c82701c/playlist.m3u8'
}

# Create the output directory if it doesn't exist
os.makedirs('data/movies', exist_ok=True)

for movie_name, url in movies.items():
    output_file = f'data/movies/{movie_name}.mp3'
    
    # Construct the ffmpeg command
    command = f'ffmpeg -protocol_whitelist file,http,https,tcp,tls,crypto -i "{url}" -vn -acodec pcm_s16le -ar 44100 -ac 2 "{output_file}"'
    
    # Execute the command
    os.system(command)
    
    print(f"Processed {movie_name}")
    
    # Optionally, display an audio player for the processed file
    # display(Audio(filename=output_file))

ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-7)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716729489913/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716729489913/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716729489913/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716729489913/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716729489913/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libo

# Test whisper ASR

In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "/home/ec2-user/SageMaker/efs/Models/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s=30,
    batch_size=16,  # batch size for inference - set based on your device
    torch_dtype=torch_dtype,
    device=device,
)

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]
sample = './data/movies/foreigner_10m_20m.mp3'

result = pipe(sample)
print("### Result ###")
print(result["text"])



/home/ec2-user/SageMaker/efs/conda_envs/whisper/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Generating validation split: 100%|██████████| 1/1 [00:02<00:00,  2.40s/ examples]
/home/ec2-user/SageMaker/efs/conda_envs/whisper/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug f

 Absolutely must be signed now. What's the latest reaction on social media? Same as before. Fear and anger. Anger directed at who? From the Prime Minister on down. CNN and the Irish Times want interviews with you. You can do tonight or tomorrow morning at 8. Tomorrow morning by phone. What else? You've a call waiting on line 2. Waiting? For 30 minutes now. A Mr. Kwan from London, very insistent. What does he want? His daughter was killed in the bombing. I tried my best to console him, but he really just wants to speak to you. Saw you on TV. Oh, Christ, Beth. Civic duty. Take it and be done. This is Liam Hennessey. How may I help you? What on earth makes you think I know who killed your daughter? Because you are the first deputy minister and advisor to Kunshan and a former UDI member. I'm truly sad for your loss, Mr. Kwan. I have a daughter myself, and I condemn these acts of violence, but I can't help you. Then please tell me someone whom I know the names of the bombers, Mr. Hennessy. 

In [7]:
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 32.5 MB/s eta 0:00:00a 0:00:01


In [10]:
import nltk
nltk.download('punkt_tab')  # Download the required data (one-time)

def split_into_sentences_nltk(text):
    sentences = nltk.sent_tokenize(text)
    return '\n'.join(sentences)

# Example usage
text = split_into_sentences_nltk(result["text"])
print(text)

 Absolutely must be signed now.
What's the latest reaction on social media?
Same as before.
Fear and anger.
Anger directed at who?
From the Prime Minister on down.
CNN and the Irish Times want interviews with you.
You can do tonight or tomorrow morning at 8.
Tomorrow morning by phone.
What else?
You've a call waiting on line 2.
Waiting?
For 30 minutes now.
A Mr. Kwan from London, very insistent.
What does he want?
His daughter was killed in the bombing.
I tried my best to console him, but he really just wants to speak to you.
Saw you on TV.
Oh, Christ, Beth.
Civic duty.
Take it and be done.
This is Liam Hennessey.
How may I help you?
What on earth makes you think I know who killed your daughter?
Because you are the first deputy minister and advisor to Kunshan and a former UDI member.
I'm truly sad for your loss, Mr. Kwan.
I have a daughter myself, and I condemn these acts of violence, but I can't help you.
Then please tell me someone whom I know the names of the bombers, Mr. Hennessy.


[nltk_data] Downloading package punkt_tab to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [11]:
with open("./data/movies/trainscripts_foreigner_10m_20m.txt", 'w') as f:
    f.write(text)

In [4]:
print(".\n".join(result["text"].split(". ")))

 Absolutely must be signed now.
What's the latest reaction on social media? Same as before.
Fear and anger.
Anger directed at who? From the Prime Minister on down.
CNN and the Irish Times want interviews with you.
You can do tonight or tomorrow morning at 8.
Tomorrow morning by phone.
What else? You've a call waiting on line 2.
Waiting? For 30 minutes now.
A Mr.
Kwan from London, very insistent.
What does he want? His daughter was killed in the bombing.
I tried my best to console him, but he really just wants to speak to you.
Saw you on TV.
Oh, Christ, Beth.
Civic duty.
Take it and be done.
This is Liam Hennessey.
How may I help you? What on earth makes you think I know who killed your daughter? Because you are the first deputy minister and advisor to Kunshan and a former UDI member.
I'm truly sad for your loss, Mr.
Kwan.
I have a daughter myself, and I condemn these acts of violence, but I can't help you.
Then please tell me someone whom I know the names of the bombers, Mr.
Hennessy.
